# Integrating LangChain and Hugging Face

It's very useful to [use Hugging Face models within LangChain](https://python.langchain.com/docs/integrations/providers/huggingface/). Let's see how.

## Install required packages

In [1]:
%pip install langchain langchain_community
#For API Calls
%pip install transformers
%pip install accelerate
%pip install bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 31.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 25.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.9/438.9 kB 28.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.6 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.65
    Uninstalling langchain-core-0.3.65:
      Successfully uninstalled langchain-core-0.3.65
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.25
    Uninstalling langchain-0.3.25:
      Successfully uninstalled langchain-0.3.25
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 126.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 96.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━

In [2]:
from langchain import PromptTemplate, HuggingFaceHub, LLMChain
import os

from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv()

openai_key = os.getenv("OPENAI_API_KEY")
hf_token = os.getenv("HUGGINGFACEHUB_API_TOKEN")


## Approach 1:  Access Models Hosted on Hugging Face Through API

In [3]:
from huggingface_hub import login
login()

In [4]:
%pip install langchain-huggingface -q

In [ ]:
os.environ["HUGGINGFACEHUB_API_TOKEN"] = ""  # your valid token

In [6]:
from langchain.prompts import PromptTemplate
from langchain_huggingface.llms import HuggingFaceEndpoint

# 1. Define prompt
prompt = PromptTemplate(
    input_variables=["product"],
    template="What is a good name for a company that makes {product}?"
)

# 2. Define LLM with model kwargs directly
llm = HuggingFaceEndpoint(
    repo_id="deepseek-ai/DeepSeek-R1",
    # huggingfacehub_api_token="HF_TOKEN", # already provided
    provider="hf-inference"
)

# 3. construct chain
chain = LLMChain(prompt=prompt, llm=llm)

# 4. run it
response = chain.run("eco-friendly paint")
print(response)

/tmp/ipython-input-6-2646358833.py:17: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(prompt=prompt, llm=llm)
/tmp/ipython-input-6-2646358833.py:18: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = chain.run("eco-friendly paint")
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


HfHubHTTPError: 402 Client Error: Payment Required for url: https://router.huggingface.co/hf-inference/models/deepseek-ai/DeepSeek-R1 (Request ID: Root=1-685badd3-4dc770941635a16014e3833b;315c78c2-bd4d-4ab3-aba2-ce75fcba0b03)

You have exceeded your monthly included credits for Inference Providers. Subscribe to PRO to get 20x more monthly included credits.

Damn, I'm broke - I did the Agent course and spent all my tokens there. But there is a solution:

## Approach 02: Download Model Locally (Create Pipelines)

For this, the use of a GPU is heavily recommended. Intel CPUs are supported, but they will run much slower.

In [ ]:
from langchain.llms import HuggingFacePipeline
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, AutoModelForSeq2SeqLM

Following the [updated guide for Hugging Face on LangChain](https://python.langchain.com/docs/integrations/llms/huggingface_pipelines/), we should do:  

In [45]:
from langchain_huggingface.llms import HuggingFacePipeline

hf = HuggingFacePipeline.from_model_id(
    model_id="google/flan-t5-large",
    task="text2text-generation",
    pipeline_kwargs={
        "max_new_tokens": 128,
        "do_sample": True,
        "temperature": 0.7,
    },
    model_kwargs={},  # optional
)


Device set to use cuda:0


In [48]:
from langchain_core.prompts import PromptTemplate
from langchain.chains import LLMChain

template = """Question: {question}

Answer: Let's think step by step."""
prompt = PromptTemplate.from_template(template)

chain = LLMChain(prompt=prompt, llm=hf)

chain.run("What is electroencephalography?")


'Electroencephalography is the study of the electrical activity in the brain. Electrical activity in the brain is called electroencephalography. So the answer is electroencephalography.'

The above is equivalent to running the chain in this more modern and efficient way, with the `|` operator:

In [49]:
chain = prompt | hf

question = "What is electroencephalography?"

print(chain.invoke({"question": question}))

Electroencephalography is the study of the electrical activity of the brain. The brain is the central nervous system. Electroencephalography is a technique used to study the electrical activity of the brain. So the answer is electroencephalography.


> side note: I tried to replicate the example from earlier (see below) but the model is just echoing back the input. Maybe it's a prompt problem?

In [52]:
# to replicate the example from earlier:

from langchain_core.prompts import PromptTemplate
from langchain.chains import LLMChain

template = "Generate a name for a company that makes {product}. Answer with the name of the company"
prompt = PromptTemplate.from_template(template)

chain = LLMChain(prompt=prompt, llm=hf)

chain.run("Colorful socks")

'Colorful Socks'